In [1]:
import ee
import time
import re

# Authenticate & initialize Earth Engine
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:


def export_image_to_drive(image_id, bands, scale=30, description="GEE_Export", folder="GEE_Exports"):
    """
    Exports a single GEE image to Google Drive using the image's bounding box.

    Args:
        image_id (str): Full path to the GEE image asset
        bands (list[str]): Band names to export (e.g., ["dswe"])
        scale (int): Export pixel size (meters)
        description (str): Task name (for GEE task manager) and also safe filename
        folder (str): Google Drive folder name
    """
    image = ee.Image(image_id).select(bands)
    region = image.geometry().bounds()

    # ✅ Ensure filename prefix is flat and unique
    file_prefix = re.sub(r'\W+', '_', description)

    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"Export_{file_prefix}",   # 👈 Task name (internal to GEE)
        folder=folder,                         # 👈 Drive folder (shared across tasks)
        fileNamePrefix=file_prefix,            # 👈 Actual filename, unique per export
        region=region,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    print(f"📤 Started export → Drive/{folder}/{file_prefix}.tif  (asset: {image_id})")
    return task


def list_image_assets(parent_folder):
    """
    Lists IMAGE assets found directly under a GEE folder/collection path.

    Args:
        parent_folder (str): e.g.,
          'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products'

    Returns:
        list[dict]: Each dict is an asset metadata item containing 'name', 'type', etc.
    """
    # ee.data.listAssets expects a dict with 'parent'
    resp = ee.data.listAssets({'parent': parent_folder})
    assets = resp.get('assets', []) if isinstance(resp, dict) else []
    # Keep only IMAGE assets
    images = [a for a in assets if a.get('type') == 'IMAGE']
    # Sort by asset basename (lexicographic; works well for DSWE_YYYY_MM)
    images.sort(key=lambda a: a['name'].split('/')[-1])
    return images


def export_all_dswe_in_folder(
    parent_folder,
    bands=("dswe",),
    scale=30,
    drive_folder="GEE_DSWE",
    name_regex=None,
    monitor=False,
    sleep_seconds=10
):
    """
    Exports all DSWE image assets in a folder to Google Drive.

    Args:
        parent_folder (str): GEE asset folder containing DSWE images
        bands (tuple|list): Bands to export (default: ('dswe',))
        scale (int): Export pixel size in meters
        drive_folder (str): Destination folder in Google Drive
        name_regex (str|None): Optional regex to filter asset basenames (e.g., r'^DSWE_2024_')
        monitor (bool): If True, wait and poll each task until it finishes (serially)
        sleep_seconds (int): Poll interval when monitor=True
    """
    assets = list_image_assets(parent_folder)
    if not assets:
        print(f"⚠️ No IMAGE assets found under: {parent_folder}")
        return

    pattern = re.compile(name_regex) if name_regex else None
    tasks = []

    for meta in assets:
        asset_id = meta['name']  # Full path
        basename = asset_id.split('/')[-1]  # e.g., 'DSWE_2024_08'

        if pattern and not pattern.search(basename):
            continue

        # 🧼 Sanitize the filename to prevent Drive from creating subfolders
        safe_description = re.sub(r'\W+', '_', basename)

        try:
            task = export_image_to_drive(
                image_id=asset_id,
                bands=list(bands),
                scale=scale,
                description=safe_description,  # ✅ Unique, flat-safe prefix
                folder=drive_folder
            )
            tasks.append((basename, task))
        except Exception as e:
            print(f"❌ Failed to start export for {basename}: {e}")

    print(f"✅ Started {len(tasks)} export task(s). Files will appear in Drive/{drive_folder}.")

    if monitor and tasks:
        print("🔎 Monitoring tasks (this may take a while)...")
        for basename, task in tasks:
            while task.active():
                print(f"⏳ Export running: {basename} ...")
                time.sleep(sleep_seconds)
            status = task.status()
            state = status.get('state', 'UNKNOWN')
            if state == 'COMPLETED':
                print(f"✅ Done: {basename}")
            else:
                print(f"⚠️ Task ended with state={state}: {basename} :: {status}")

In [4]:
# -----------------------------
# 🚀 Example: export all DSWE products in your folder
# -----------------------------
if __name__ == "__main__":
    PARENT_FOLDER = "projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products"
    BANDS = ("dswe",)
    SCALE = 30
    DRIVE_FOLDER = "GEE_DSWE"

    # Optionally filter, e.g. only 2024: name_regex=r'^DSWE_2024_'
    NAME_REGEX = r'^DSWE_2019_'  # or r'^DSWE_2024_'

    # Set monitor=True if you want to block and watch each task until it finishes
    export_all_dswe_in_folder(
        parent_folder=PARENT_FOLDER,
        bands=BANDS,
        scale=SCALE,
        drive_folder=DRIVE_FOLDER,
        name_regex=NAME_REGEX,
        monitor=False,          # set to True to poll until completion
        sleep_seconds=15
    )

📤 Started export → Drive/GEE_DSWE/DSWE_2019_01.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_01)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_02.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_02)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_03.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_03)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_04.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_04)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_05.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_05)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_06.tif  (asset: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2019_06)
📤 Started export → Drive/GEE_DSWE/DSWE_2019_07.tif  (asset: proj